In [1]:
import pandas as pd
import numpy as np
import nfl_data_py as nfl
import seaborn as sns
import matplotlib.pyplot as plt
import os

In [2]:
chap_2_file = "./data/pbp_py_chap_2_2.csv"

if os.path.isfile(chap_2_file):
    pbp_py = pd.read_csv(chap_2_file, low_memory=False)
else:
    pbp_py = nfl.import_pbp_data([2025])
    pbp_py.to_csv(chap_2_file)

In [3]:
pbp_py_p_s = pbp_py.groupby(["passer_id", "passer"]).agg(
    {"passing_yards": ["mean", "count", "sum"]}
)

In [4]:
pbp_py_p_s.columns = list(map("_".join, pbp_py_p_s.columns.values))

In [6]:
pbp_py_p_s.rename(
    columns={"passing_yards_mean": "ypa", "passing_yards_count": "n", "passing_yards_sum": "sum"}, inplace=True
)

In [9]:
pbp_py_p_s.sort_values(by=["ypa"], ascending=False).head()

,,ypa,n,sum
passer_id,passer,,,
00-0035645,G.Olszewski,33.0,1,33.0
00-0036414,C.Akers,32.0,1,32.0
00-0036259,J.Jennings,29.0,1,29.0
00-0033357,T.Hill,28.5,2,57.0
00-0033400,M.Haack,28.0,1,28.0


In [11]:
pbp_py_p_s_50 = pbp_py_p_s.query("n >= 50").sort_values(by=["ypa"], ascending=False)
pbp_py_p_s_50.head()

,,ypa,n,sum
passer_id,passer,,,
00-0034796,L.Jackson,13.276042,192,2549.0
00-0039851,D.Maye,12.509044,387,4841.0
00-0034869,S.Darnold,12.453731,335,4172.0
00-0026498,M.Stafford,12.196759,432,5269.0
00-0032268,M.Mariota,12.194245,139,1695.0


In [41]:
pbp_py["pass_length_air_yards"] = np.where(
    pbp_py["air_yards"] >= 20, "long", "short"
)
pbp_py_p_s_pl = pbp_py.groupby(
    ["passer_id", "passer", "pass_length_air_yards"]
).agg({"passing_yards": ["mean", "count", "sum"]})

In [42]:
pbp_py_p_s_pl.columns = list(map("_".join, pbp_py_p_s_pl.columns.values))
pbp_py_p_s_pl.rename(
    columns={"passing_yards_mean": "ypa", "passing_yards_count": "n", "passing_yards_sum": "sum"}, inplace=True
)

In [43]:
pbp_py_p_s_pl.reset_index(inplace=True)

In [61]:
q_value = (
    "(n >= 50 & "
    + 'pass_length_air_yards == "short") | '
    + "(n >= 5 & "
    + 'pass_length_air_yards == "long")'
)
pbp_py_p_s_pl = pbp_py_p_s_pl.query(q_value).reset_index()

In [62]:
cols_save = ["passer_id", "passer", "pass_length_air_yards", "ypa"]
air_yards_py = pbp_py_p_s_pl[cols_save].copy()

In [63]:
print(
    pbp_py_p_s_pl[["pass_length_air_yards", "passer", "ypa"]]
    .sort_values(["passer", "pass_length_air_yards"])
    .to_string()
)

   pass_length_air_yards        passer        ypa
1                   long     A.Rodgers  34.375000
2                  short     A.Rodgers   8.896341
79                 short        B.Cook   8.069767
28                  long    B.Mayfield  35.619048
29                 short    B.Mayfield   9.145963
66                  long         B.Nix  30.896552
67                 short         B.Nix   8.607792
57                  long       B.Purdy  27.571429
58                 short       B.Purdy   9.521531
60                  long       B.Young  31.950000
61                 short       B.Young   8.642623
62                  long      C.Stroud  35.315789
63                 short      C.Stroud   9.600000
82                  long        C.Ward  30.954545
83                 short        C.Ward   8.265781
16                  long       C.Wentz  35.571429
17                 short       C.Wentz   9.388350
74                  long    C.Williams  29.358974
75                 short    C.Williams  10.103550


In [60]:
pbp_py[(pbp_py.passer == 'L.Jackson') & (pbp_py.pass_length_air_yards == "long")].loc[203].air_yards

22.0

In [64]:
pbp_py_p_s_pl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89 entries, 0 to 88
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   index                  89 non-null     int64  
 1   passer_id              89 non-null     object 
 2   passer                 89 non-null     object 
 3   pass_length_air_yards  89 non-null     object 
 4   ypa                    89 non-null     float64
 5   n                      89 non-null     int64  
 6   sum                    89 non-null     float64
dtypes: float64(2), int64(2), object(3)
memory usage: 5.0+ KB


In [65]:
len(pbp_py_p_s_pl.passer_id.unique())

50

In [66]:
sns.lmplot(data=pbp_py_p_s_pl, x="ypa", y="ypa_last", col="pass_length_air_yards")
plt.show()

KeyError: "['ypa_last'] not in index"